In [ ]:
import praw
import pandas as pd
from datetime import datetime

SECRET = "fDTjgQd2sS6gmVrqm7IcPPhp05NHXg"
APP_ID = "KGv2MMUcdlqidiWTIP4YPA"

reddit = praw.Reddit(
        client_id=APP_ID,
        client_secret=SECRET,
        user_agent="Comment Extraction")

search_query = "iPhone"
max_comments = 1000

comment_bodies = []
comment_authors = []
comment_upvotes = []
comment_awards = []
comment_dates = []

start_date = datetime(2022, 1, 1)
end_date = datetime(2023, 11, 2)

for submission in reddit.subreddit("all").search(search_query, limit=max_comments):
    submission.comments.replace_more(limit=0)
    for comment in submission.comments:
        if comment.is_root:
            comment_date = datetime.utcfromtimestamp(comment.created_utc)
            if start_date <= comment_date <= end_date:
                comment_bodies.append(comment.body)
                comment_authors.append(comment.author.name if comment.author else "[deleted]")
                comment_upvotes.append(comment.score)
                comment_awards.append(comment.total_awards_received)
                comment_dates.append(comment_date)

data = {
    "Comment_Body": comment_bodies,
    "Author": comment_authors,
    "Upvotes": comment_upvotes,
    "Awards": comment_awards,
    "Date": comment_dates,
}

temp_df = pd.DataFrame(data)

temp_df = temp_df[temp_df['Author'] != 'AutoModerator']
temp_df = temp_df[temp_df['Comment_Body'] != '[deleted]']
temp_df = temp_df[~temp_df['Comment_Body'].str.contains(r'\[Banned for overrating\]', case=False)]
temp_df = temp_df[~temp_df['Comment_Body'].str.contains(r'\br/\w+', case=False)]
temp_df = temp_df[temp_df['Comment_Body'].str.contains("iPhone", case=False)]
temp_df = temp_df.dropna(subset=['Comment_Body'])
temp_df['Search'] = 'iPhone'
df = pd.concat([df, temp_df], ignore_index=True)
df = df.dropna(subset=['Comment_Body']).drop_duplicates(subset=['Comment_Body'])
df.to_csv('reddit_comments.csv', index=False)


df